# Python Flask REST API Prototype V 1.0.3  #

*스토어 프로시저 -> 년월 평균 실측파고* 


In [18]:
'''
CREATE TABLE bouy (
  num int(11) NOT NULL AUTO_INCREMENT,
  obs_post_id char(7) NOT NULL,
  obs_post_name varchar(30) NOT NULL,
  obs_lat varchar(10) NOT NULL,
  obs_lon varchar(10) NOT NULL,
  obs_last_req_cnt varchar(11) NOT NULL,
  record_time varchar(19) NOT NULL,
  water_temp varchar(10) NOT NULL,
  salinity varchar(10) NOT NULL,
  air_temp varchar(10) NOT NULL,
  air_pres varchar(10) NOT NULL,     
  wind_dir varchar(10) NOT NULL,
  wind_speed varchar(10) NOT NULL,
  current_dir varchar(10) NOT NULL,
  current_speed varchar(10) NOT NULL,
  wave_height varchar(10) NOT NULL,
  posted datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;
END'''

'\nCREATE TABLE bouy (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  obs_post_id char(7) NOT NULL,\n  obs_post_name varchar(30) NOT NULL,\n  obs_lat varchar(10) NOT NULL,\n  obs_lon varchar(10) NOT NULL,\n  obs_last_req_cnt varchar(11) NOT NULL,\n  record_time varchar(19) NOT NULL,\n  water_temp varchar(10) NOT NULL,\n  salinity varchar(10) NOT NULL,\n  air_temp varchar(10) NOT NULL,\n  air_pres varchar(10) NOT NULL,     \n  wind_dir varchar(10) NOT NULL,\n  wind_speed varchar(10) NOT NULL,\n  current_dir varchar(10) NOT NULL,\n  current_speed varchar(10) NOT NULL,\n  wave_height varchar(10) NOT NULL,\n  posted datetime DEFAULT NOW(),\n  primary key (num)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;\nEND'

In [37]:
'''
DELIMITER //
CREATE PROCEDURE AveragemeasuredValue(
	IN year VARCHAR(255),
	IN month VARCHAR(255)
)
BEGIN
	select
  	obs_post_name as area, substr(record_time,1,4) as year, substr(record_time,6,2) as month,
  	round(avg(replace(water_temp,'None',0)),1) as water_temp,
  	round(avg(replace(salinity,'None',0)),1) as salinity, 
  	round(avg(replace(air_temp,'None',0)),1) as air_temp, 
  	round(avg(replace(air_pres,'None',0)),1) as air_pres, 
  	round(avg(replace(wind_dir,'None',0)),1) as wind_dir, 
	round(avg(replace(wind_speed,'None',0)),1) as wind_speed, 
	round(avg(replace(current_dir,'None',0)),1) as current_dir, 
	round(avg(replace(current_speed,'None',0)),1) as current_speed, 
	round(avg(replace(wave_height,'None',0)),1) as wave_height from bouy 
	where substr(record_time,1,4) = year
	and substr(record_time,6,2) = month group by obs_post_name;
END //
DELIMITER ;
END'''

"\nDELIMITER //\nCREATE PROCEDURE AveragemeasuredValue(\n\tIN year VARCHAR(255),\n\tIN month VARCHAR(255)\n)\nBEGIN\n\tselect\n  \tobs_post_name as area, substr(record_time,1,4) as year, substr(record_time,6,2) as month,\n  \tround(avg(replace(water_temp,'None',0)),1) as water_temp,\n  \tround(avg(replace(salinity,'None',0)),1) as salinity, \n  \tround(avg(replace(air_temp,'None',0)),1) as air_temp, \n  \tround(avg(replace(air_pres,'None',0)),1) as air_pres, \n  \tround(avg(replace(wind_dir,'None',0)),1) as wind_dir, \n\tround(avg(replace(wind_speed,'None',0)),1) as wind_speed, \n\tround(avg(replace(current_dir,'None',0)),1) as current_dir, \n\tround(avg(replace(current_speed,'None',0)),1) as current_speed, \n\tround(avg(replace(wave_height,'None',0)),1) as wave_height from bouy \n\twhere substr(record_time,1,4) = year\n\tand substr(record_time,6,2) = month group by obs_post_name;\nEND //\nDELIMITER ;\nEND"

In [38]:
'''
call AveragemeasuredValue('2020','08')
END'''

"\ncall AveragemeasuredValue('2020','08')\nEND"

In [19]:
!sudo pip3 install pymysql

In [20]:
!sudo pip3 install flask-ngrok

In [21]:
!sudo pip3 install flask_restful

In [22]:
import json
import pymysql
import configparser
from flask import Flask
from decimal import Decimal
from datetime import datetime
from google.colab import drive
from flask_restful import reqparse
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok

In [23]:
c=drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
CNF_INI = 'drive/My Drive/development/config/openapi.ini'
ERR_URI = 'drive/My Drive/development/test/error.log'

In [25]:
config = configparser.ConfigParser()
config.read(CNF_INI)
HOST = config['mysql']['host']
USER = config['mysql']['user']
PASSWORD = config['mysql']['password']
DATABASE = config['mysql']['database']
PORT = config['mysql']['port']
CHARSET = config['mysql']['charset']

In [26]:
def default(obj):
  if isinstance(obj, Decimal):
    return str(obj)
  raise TypeError("Object of type '%s' is not JSON serializable" % type(obj).__name__)

In [27]:
def db_access():
  conn = pymysql.connect(
      host=HOST,
      user=USER,
      password=PASSWORD,
      db=DATABASE,
      charset=CHARSET,
      port=int(PORT),
      cursorclass=pymysql.cursors.DictCursor)
  return conn

In [33]:
def exec_status_bouy(conn,year,month):
  with conn.cursor() as cursor:
    query = get_status_bouy(year,month)
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [34]:
def get_status_bouy(year,month):
  query = 'select \
  obs_post_name as area, substr(record_time,1,4) as year, substr(record_time,6,2) as month, \
  round(avg(replace(water_temp,\'None\',0)),1) as water_temp, \
  round(avg(replace(salinity,\'None\',0)),1) as salinity, \
  round(avg(replace(air_temp,\'None\',0)),1) as air_temp, \
  round(avg(replace(air_pres,\'None\',0)),1) as air_pres, \
  round(avg(replace(wind_dir,\'None\',0)),1) as wind_dir, \
  round(avg(replace(wind_speed,\'None\',0)),1) as wind_speed, \
  round(avg(replace(current_dir,\'None\',0)),1) as current_dir, \
  round(avg(replace(current_speed,\'None\',0)),1) as current_speed, \
  round(avg(replace(wave_height,\'None\',0)),1) as wave_height from bouy \
  where substr(record_time,1,4) = \'{}\' \
  and substr(record_time,6,2) = \'{}\' group by obs_post_name'.format(year,month)
  print(query)
  return query 

In [67]:
def sp_status_bouy(conn,year,month):
  with conn.cursor() as cursor:
    store_procedure = 'call AveragemeasuredValue(\'{}\',\'{}\')'.format(year,month)
    cursor.execute(store_procedure)
    data = cursor.fetchall()
    return data  

In [68]:
class confirm_api(Resource):
  def get(self):
    return {'status':'200'}

In [69]:
class status_bouy(Resource):
  def get(self):
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('year', default='2020', type=str)
      parser.add_argument('month', default='08', type=str)
      args = parser.parse_args()
      
      year = args['year']
      month = args['month']     

      #conn = db_access()
      #data = exec_status_bouy(conn,year,month)
      conn = db_access()
      data = sp_status_bouy(conn,year,month)

      print(data)
      
      if data:
        return {'staus': '101', 'message': 'I have a data', 'data': data}  
      else:
        return {'staus': '100', 'message': 'There are no data.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))   

## Howto ##

통계 쿼리 : 년, 월 평균 온습도 /bouy/status?year=2020&month=07

통계 쿼리 : 년, 월 평균 실측파고 /bouy/status?&year=2020&month=07


In [71]:
app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)
api.add_resource(confirm_api,'/')
api.add_resource(status_bouy,'/bouy/status/')
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://26dd2afd43d4.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Aug/2020 02:56:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2020 02:56:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2020 02:56:53] "GET /bouy/status HTTP/1.1" 308 -
127.0.0.1 - - [26/Aug/2020 02:56:54] "GET /bouy/status/ HTTP/1.1" 200 -


[{'area': '감천항', 'year': '2020', 'month': '08', 'water_temp': 16.1, 'salinity': 0.0, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 151.4, 'current_speed': 0.1, 'wave_height': 0.1}, {'area': '경인항', 'year': '2020', 'month': '08', 'water_temp': 23.6, 'salinity': 14.1, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 217.7, 'current_speed': 1.6, 'wave_height': 0.0}, {'area': '경포대해수욕장', 'year': '2020', 'month': '08', 'water_temp': 22.7, 'salinity': 29.7, 'air_temp': 25.3, 'air_pres': 1008.0, 'wind_dir': 191.3, 'wind_speed': 3.6, 'current_dir': 203.0, 'current_speed': 0.4, 'wave_height': 0.4}, {'area': '광양항', 'year': '2020', 'month': '08', 'water_temp': 22.3, 'salinity': 16.6, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 172.5, 'current_speed': 0.5, 'wave_height': 0.0}, {'area': '군산항', 'year': '2020', 'month': '08', 'water_temp': 24.4, 'salinity': 0.0, 'air_temp': 0.0, 'air_pres

127.0.0.1 - - [26/Aug/2020 02:57:17] "GET /bouy/status HTTP/1.1" 308 -
127.0.0.1 - - [26/Aug/2020 02:57:17] "GET /bouy/status/ HTTP/1.1" 200 -


[{'area': '감천항', 'year': '2020', 'month': '08', 'water_temp': 16.1, 'salinity': 0.0, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 151.4, 'current_speed': 0.1, 'wave_height': 0.1}, {'area': '경인항', 'year': '2020', 'month': '08', 'water_temp': 23.6, 'salinity': 14.1, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 217.7, 'current_speed': 1.6, 'wave_height': 0.0}, {'area': '경포대해수욕장', 'year': '2020', 'month': '08', 'water_temp': 22.7, 'salinity': 29.7, 'air_temp': 25.3, 'air_pres': 1008.0, 'wind_dir': 191.3, 'wind_speed': 3.6, 'current_dir': 203.0, 'current_speed': 0.4, 'wave_height': 0.4}, {'area': '광양항', 'year': '2020', 'month': '08', 'water_temp': 22.3, 'salinity': 16.6, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 172.5, 'current_speed': 0.5, 'wave_height': 0.0}, {'area': '군산항', 'year': '2020', 'month': '08', 'water_temp': 24.4, 'salinity': 0.0, 'air_temp': 0.0, 'air_pres

127.0.0.1 - - [26/Aug/2020 02:57:39] "GET /bouy/status?year=2020&month=07 HTTP/1.1" 308 -
127.0.0.1 - - [26/Aug/2020 02:57:40] "GET /bouy/status/?year=2020&month=07 HTTP/1.1" 200 -


[{'area': '감천항', 'year': '2020', 'month': '07', 'water_temp': 17.2, 'salinity': 0.0, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 191.6, 'current_speed': 0.1, 'wave_height': 0.1}, {'area': '경인항', 'year': '2020', 'month': '07', 'water_temp': 23.5, 'salinity': 19.7, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 237.2, 'current_speed': 1.5, 'wave_height': 0.0}, {'area': '경포대해수욕장', 'year': '2020', 'month': '07', 'water_temp': 20.4, 'salinity': 30.9, 'air_temp': 18.2, 'air_pres': 1007.8, 'wind_dir': 183.8, 'wind_speed': 3.9, 'current_dir': 178.5, 'current_speed': 0.4, 'wave_height': 0.9}, {'area': '광양항', 'year': '2020', 'month': '07', 'water_temp': 22.0, 'salinity': 23.0, 'air_temp': 0.0, 'air_pres': 0.0, 'wind_dir': 0.0, 'wind_speed': 0.0, 'current_dir': 154.7, 'current_speed': 0.5, 'wave_height': 0.0}, {'area': '군산항', 'year': '2020', 'month': '07', 'water_temp': 22.2, 'salinity': 0.0, 'air_temp': 0.0, 'air_pres